# Test Cases for Memory

In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

import uuid
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import MemorySaver
from email_assistant.email_assistant_hitl_memory import overall_workflow

/Users/rlm/Desktop/Code/interrupt_workshop


## Accept `write_email` and `schedule_meeting`

Our first test examines what happens when a user accepts the agent's actions without modification. This baseline case helps us understand how the system behaves when no feedback is provided:

1. We'll use the same tax planning email from our previous tests
2. The system will classify it as "RESPOND" and propose scheduling a meeting
3. We'll accept the meeting schedule without changes
4. The agent will generate an email confirming the meeting
5. We'll accept the email without changes

This test demonstrates the default behavior of our memory-enabled system. When a user simply accepts proposed actions, we expect minimal or no memory updates since there's no explicit feedback to learn from. However, the system will still leverage existing memory (if any) when generating its responses.

In [2]:
from langgraph.types import Command

# Helper function to display memory content
def display_memory_content(store, namespace=None):
    # Display current memory content for all namespaces
    print("\n======= CURRENT MEMORY CONTENT =======")
    if namespace:
        memory = store.get(namespace, "user_preferences")
        if memory:
            print(f"\n--- {namespace[1]} ---")
            print({"preferences": memory.value})
        else:
            print(f"\n--- {namespace[1]} ---")
            print("No memory found")
    else:
        for namespace in [
            ("email_assistant", "triage_preferences"),
            ("email_assistant", "response_preferences"),
            ("email_assistant", "cal_preferences"),
            ("email_assistant", "background")
        ]:
            memory = store.get(namespace, "user_preferences")
            if memory:
                print(f"\n--- {namespace[1]} ---")
                print({"preferences": memory.value})
            else:
                print(f"\n--- {namespace[1]} ---")
                print("No memory found")
            print("=======================================\n")

In [25]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_1 = uuid.uuid4()
thread_config_1 = {"configurable": {"thread_id": thread_id_1}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt
display_memory_content(store)

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant inform

Accept the schedule_meeting tool call

As we examine the initial `schedule_meeting` proposal, note how the system uses existing memory to inform its decisions:

1. The default calendar preferences show a preference for 30-minute meetings, though the email requests 45 minutes
2. The agent still proposes a 45-minute meeting, respecting the sender's specific request
3. We accept this proposal without modification to see if simple acceptance triggers any memory updates

After running this step, we'll check the memory contents to confirm whether acceptance alone leads to memory updates. Simple acceptance represents the baseline user experience - the system works as intended without requiring adjustments.

In [26]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user accepting the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hello,\n\nThank you for reaching out. I am available on Tuesday, April 22nd at 2:00 PM for a 45-minute call to discuss tax planning strategies. I have scheduled the meeting accordingly.\n\nLooking forward to your suggestions and our discussion.\n\nBest regards,\nLance'}}


Accept the write_email tool call

Now we'll accept the email draft that confirms the meeting scheduling:

1. The email draft is generated with knowledge of our calendar preferences
2. It includes details about the meeting time, duration, and purpose
3. We'll accept it without changes to complete the baseline test case

After accepting, we'll check all memory stores to see if any updates occurred. As expected, simply accepting the agent's proposals doesn't provide strong learning signals - there's no clear feedback about what the user likes or dislikes about the agent's approach.

The trace link shows the complete workflow execution, where we can see that the memory is used in the LLM call for response generation, but no memory updates occur, which is the expected behavior for simple acceptances.

In [27]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting the write_email tool call
display_memory_content(store)


Simulating user accepting the write_email tool call...
================================== Ai Message ==================================
Tool Calls:
  Done (call_02WPNb2qpXXVx43d3Z0VzWqv)
 Call ID: call_02WPNb2qpXXVx43d3Z0VzWqv
  Args:
    done: True

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR Department deadline reminders\n- Subscription status / renewal reminders\

We can look at the full messages, and the trace: 

https://smith.langchain.com/public/86ff6474-29fe-452e-8829-b05a91b458eb/r

You'll notice that memory is used in the LLM call to respond. 

But the memory store *not* updated, because we haven't added any feedback via HITL.

In [28]:
state = graph.get_state(thread_config_1)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_6ZbPjmSN8h1bfl3tDCx5ssoj)
 Call ID: call_6ZbPjmSN8h1bfl3tDCx5ssoj
  Args:
    day: 2025-04-22
  check_calendar_availability (call_8M30JVmN7Wj2VlIcDqBE1995)
 Call ID: call_8M30JVmN7Wj2VlIcDqBE1995
  Args:
    day: 2025-04-24
================================= Tool Message =================================

Ava

## Edit `write_email` and `schedule_meeting`

This test explores how the system learns from direct edits to its proposed actions. When users modify the agent's suggestions, it creates clear, specific learning signals about their preferences:

1. We'll use the same tax planning email as before
2. When the agent proposes a 45-minute meeting, we'll edit it to:
   - Change the duration to 30 minutes (matching our stored preference)
   - Make the subject line more concise
3. When the agent drafts an email, we'll edit it to be:
   - Shorter and less formal
   - Structured differently

Edits provide the most explicit feedback about user preferences, letting the system learn exactly what changes are desired. We expect to see specific, targeted updates to our memory stores that reflect these edits.

In [29]:
# Same email as before
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph with new thread
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_2 = uuid.uuid4()
thread_config_2 = {"configurable": {"thread_id": thread_id_2}}

# Run the graph until the first interrupt - will be classified as "respond" and the agent will create a write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_2):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt
display_memory_content(store,("email_assistant", "cal_preferences"))

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}

======= CURRENT MEMORY CONTENT =======

--- cal_preferences ---
{'preferences': '\n30 minute meetings are preferred, but 15 minute meetings are also acceptable.\n'}


Edit the `schedule_meeting` tool call

When we edit the meeting proposal, we're providing direct, explicit feedback about our preferences. This creates a significant learning opportunity for the system:

1. The agent initially proposes a 45-minute meeting (the duration requested in the email)
2. We edit it to 30 minutes and simplify the subject from "Tax Planning Strategies Discussion" to "Tax Planning Discussion"
3. This creates clear, specific feedback about our time preferences and naming conventions

After the edit, we'll check the calendar preferences memory store to see how it's updated. The memory update should capture both:
- Our preference for shorter 30-minute meetings
- Our preference for more concise meeting subjects

The trace reveals the precise memory update logic, showing how the system analyzes the difference between its proposal and our edits to extract meaningful patterns and preferences. We can see the detailed justification for each memory update, ensuring transparency in the learning process.

In [30]:
# Now simulate user editing the schedule_meeting tool call
print("\nSimulating user editing the schedule_meeting tool call...")
edited_schedule_args = {
    "attendees": ["pm@client.com", "lance@company.com"],
    "subject": "Tax Planning Discussion",
    "duration_minutes": 30, # Changed from 45 to 30
    "preferred_day": "2025-04-22",
    "start_time": 14 # 2pm
}
for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_schedule_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after editing schedule_meeting
print("\nChecking memory after editing schedule_meeting:")
display_memory_content(store,("email_assistant", "cal_preferences"))


Simulating user editing the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hello,\n\nThank you for reaching out regarding tax planning strategies for this year. I have scheduled a 30-minute call for us on Tuesday, April 22nd at 2:00 PM to discuss your suggestions and review potential savings opportunities. If you need a longer duration, please let me know and I can adjust the meeting accordingly.\n\nLooking forward to our discussion.\n\nBest regards,\nLance'}}

Checking memory after editing schedule_meeting:

======= CURRENT MEMORY CONTENT =======

--- cal_preferences ---
{'preferences': "30 minute meetings are preferred, but 15 minute meetings are also acceptable. User prefers 30 minute meetings over longer durations such as 45 minutes when possible. User prefers concise meeting subjects (e.g., 'Tax Planning Discussion' instead of 'Tax Planning Stra

Looking at the memory after editing the calendar invitation, we can see that it's been updated with remarkably specific preferences:

1. The system has identified that we prefer 30-minute meetings over longer durations
2. It's also captured our preference for concise meeting subjects

What's particularly impressive about this memory update is:
- It doesn't just record our specific edit, but generalizes to a broader preference pattern
- It preserves all existing memory content while adding the new information
- It extracts multiple preference signals from a single edit interaction

Now, let's edit the email draft to see how the system captures different types of communication preferences:

In [ ]:
display_memory_content(store,("email_assistant", "response_preferences"))
# Now simulate user editing the write_email tool call
print("\nSimulating user editing the write_email tool call...")
edited_email_args = {
    "to": "pm@client.com",
    "subject": "Re: Tax season let's schedule call",
    "content": "Thanks! I scheduled a 30-minute call next Thursday at 3:00 PM. Would that work for you?\n\nBest regards,\nLance Martin"
}
for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_email_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after editing write_email
print("\nChecking memory after editing write_email:")
display_memory_content(store,("email_assistant", "response_preferences"))

Our email edit reveals even more sophisticated learning capabilities:

1. We've dramatically shortened and simplified the email content
2. We've changed the tone to be more casual
3. We've added a question asking for confirmation rather than assuming the time works
4. We've slightly altered the meeting details (day and time)

Looking at the updated memory, we can see that the system has extracted a key insight about our communication style:

```
When scheduling a meeting, ask the recipient to confirm if the proposed time works for them, rather than assuming and stating the meeting is already scheduled.
```

This demonstrates the system's ability to:
- Analyze our edit not just at a superficial level, but to understand intent
- Extract generalizable principles from specific examples
- Preserve all existing guidance while adding new insights
- Maintain the organization and structure of the memory

These targeted, high-quality memory updates will improve all future interactions without requiring repeated corrections.

In [32]:
state = graph.get_state(thread_config_2)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_3lgcL8JNtxaakOR8NqZyVH2n)
 Call ID: call_3lgcL8JNtxaakOR8NqZyVH2n
  Args:
    day: 2025-04-22
  check_calendar_availability (call_Ru9AMMrCf7wlWPcS9gV7ohCq)
 Call ID: call_Ru9AMMrCf7wlWPcS9gV7ohCq
  Args:
    day: 2025-04-24
================================= Tool Message =================================

Ava

## Ignore `write_email`, `schedule_meeting`, and `question`

This test explores how the system learns from rejection. When users ignore (reject) the agent's suggestions, it creates a strong signal about content they don't want to handle:

1. We'll first test ignoring a `schedule_meeting` request entirely
2. Then we'll test accepting a meeting but ignoring the follow-up email
3. Finally, we'll test ignoring a `question` for a different email context

These rejection signals help the system learn what types of emails and actions a user prefers not to deal with, leading to more appropriate triage decisions in the future. We expect significant updates to the triage preferences memory after each ignore action.

In [9]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_3 = uuid.uuid4()
thread_config_3 = {"configurable": {"thread_id": thread_id_3}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_3):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt
display_memory_content(store, ("email_assistant", "triage_preferences"))

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant inform

Ignore the `schedule_meeting` tool call

When we ignore a meeting scheduling request, we're signaling that we don't want to handle this type of email through the assistant. This creates a powerful learning opportunity about our triage preferences:

1. The assistant initially classified the tax planning email as "RESPOND"
2. But by ignoring the scheduling request, we indicate we'd prefer not to handle this type of email
3. The system needs to update its triage classification preferences to reflect this rejection

After ignoring the request, we'll check the triage preferences memory to see how the rejection affected the system's understanding. The memory update should show a new pattern added to the "not worth responding to" section, specifically about tax planning calls or similar recurring discussions.

The trace shows how the system processes this rejection, identifies the pattern, and updates the memory with specific justification for why this type of email should be classified differently in the future.

In [10]:
print(f"\nSimulating user ignoring the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_3):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after ignoring first tool call
print("\nChecking memory after ignoring first tool call:")
display_memory_content(store, ("email_assistant", "triage_preferences"))


Simulating user ignoring the schedule_meeting tool call...

Checking memory after ignoring first tool call:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "Emails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n- Client requests to schedule tax planning calls (unless explicitly instructed otherwise)\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR Department deadline reminders\n- Subscription status / renewal reminders\n- GitHub notifications\n\nEmails that are worth resp

Looking at the memory update after ignoring the schedule_meeting tool call, we can see a remarkable triage preference update:

1. The system has added "Client requests to schedule tax planning calls" to the "emails not worth responding to" section
2. It correctly identified the general pattern (scheduling routine calls) rather than overfitting to just this specific instance
3. It included the parenthetical note "(unless explicitly instructed otherwise)" to maintain flexibility

This update demonstrates the system's ability to:
- Infer general patterns from specific instances of rejection
- Update the triage filters that determine initial email classification
- Preserve the organization and priority of existing preferences
- Include appropriate qualifiers to avoid overly rigid rules

Next, let's see what happens when we accept the meeting but reject the email draft:

In [11]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_3 = uuid.uuid4()
thread_config_3 = {"configurable": {"thread_id": thread_id_3}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_3):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_3):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

print(f"\nSimulating user ignoring the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_3):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after ignoring second tool call
print("\nChecking memory after ignoring second tool call:")
display_memory_content(store, ("email_assistant", "triage_preferences"))

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}

Simulating user accepting the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hello,\n\nThank you for reaching out. I am available on Tuesday, April 22nd at 2:00 PM for a 45-minute call to discuss tax planning strategies. I have scheduled the meeting accordingly.\n\nLooking forward to your suggestions and the discussion.\n\nBest regards,\nLance'}}

Simulating user ignoring the write_email tool call...
================================= Tool Message =================================

User ignored this e

When we accept the meeting but ignore the email draft, we're sending a more nuanced signal about our preferences:

1. We're willing to schedule the meeting (accepting the first tool call)
2. But we don't want to send a confirmation email about it (ignoring the second tool call)

Looking at the memory update, we see another evolution of our triage preferences:

```
"Client requests to schedule routine calls (such as tax planning or similar recurring discussions)"
```

The system has:
- Broadened the pattern from just "tax planning calls" to "routine calls" generally
- Added examples in parentheses for clarity
- Positioned this in the "not worth responding to" section
- Maintained all other existing preferences

This demonstrates how the memory evolves over multiple interactions, becoming increasingly accurate and generalizable with each additional data point. The system is continuously refining its understanding based on our feedback patterns.

In [13]:
state = graph.get_state(thread_config_3)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_l9Jn2ghL4T0aPuMhjpXf36Ga)
 Call ID: call_l9Jn2ghL4T0aPuMhjpXf36Ga
  Args:
    day: 2025-04-22
  check_calendar_availability (call_8eOXPjqVpaIRDkABD80XRYUU)
 Call ID: call_8eOXPjqVpaIRDkABD80XRYUU
  Args:
    day: 2025-04-24
================================= Tool Message =================================

Ava

Now let's try an email that calls the `Question` tool

For our third rejection test, we'll use a different type of email - a casual social invitation about brunch. This gives us insight into how the system learns about personal vs. professional communication preferences:

1. The system classifies this personal invitation as "RESPOND"
2. Rather than answering directly, it uses the Question tool to ask for clarification
3. We'll ignore this question, indicating we don't want to handle these types of emails through the assistant

This test shows how ignoring questions (not just actions) can also update our triage preferences. By rejecting the clarification attempt, we signal that this entire category of email doesn't warrant response through the assistant.

In [14]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Partner <partner@home.com>",
    "subject": "Meet Jim and Lisa for brunch in 3 weeks?",
    "email_thread": "Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about."
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_4 = uuid.uuid4()
thread_config_4 = {"configurable": {"thread_id": thread_id_4}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_4):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt for Question tool
display_memory_content(store, ("email_assistant", "triage_preferences"))

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'partner@home.com', 'subject': 'Re: Meet Jim and Lisa for brunch in 3 weeks?', 'content': 'That sounds like a great idea! I checked my calendar for three weeks from now (Thursday, May 8th), and I’m available at 9:00 AM, 2:00 PM, or 4:00 PM. Do any of those times work for you, Jim, and Lisa? The new place on 17th sounds perfect.\n\nLet me know what everyone prefers and I’ll confirm the reservation.'}}

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examp

Ignore the `question` tool call

When we ignore a question from the assistant about a personal social invitation, we're providing yet another type of feedback:

1. The system initially tries to get clarification before responding
2. By ignoring the question, we indicate we don't even want to engage with this type of email
3. This suggests the entire category of social invitations should be handled differently

After ignoring, we'll check the triage preferences again to see how they've evolved. We expect to see a new entry about social invitations added to the "not worth responding to" section.

The memory update justification analyzes our rejection of the question about an informal social invitation and extracts a general pattern about our preference not to handle social invitations through the assistant.

In [15]:
print(f"\nSimulating user ignoring the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_4):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after ignoring Question tool
print("\nChecking memory after ignoring Question tool:")
display_memory_content(store, ("email_assistant", "triage_preferences"))


Simulating user ignoring the write_email tool call...

Checking memory after ignoring Question tool:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "Emails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n- Social invitations from partner (e.g., brunch plans, casual meetups)\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR Department deadline reminders\n- Subscription status / renewal reminders\n- GitHub notifications\n\nEmails that are worth responding to:\n- Direct quest

Looking at the memory update after ignoring the question about brunch plans, we see another sophisticated triage preference update:

```
"Social invitations from partner (e.g., brunch plans, casual meetups)"
```

This demonstrates how the system:
1. Correctly identifies personal social invitations as a distinct category
2. Specifically notes they're from "partner" - showing it's learning to distinguish senders
3. Provides examples to clarify the pattern
4. Adds this to the "not worth responding to" section

These three ignores have collectively taught the system quite a bit about what types of emails we prefer not to handle through the assistant:
- Tax planning calls and routine client scheduling
- Social invitations from partners
- Each with appropriate specificity and generalizability

In a real-world scenario, these learned preferences would ensure that similar future emails would be classified differently, saving the user time by automatically filtering out categories they've previously rejected.

In [16]:
state = graph.get_state(thread_config_4)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Meet Jim and Lisa for brunch in 3 weeks?
**From**: Partner <partner@home.com>
**To**: Lance Martin <lance@company.com>

Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about.

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_5jTHskfgPCFQKakHl2dFxvsD)
 Call ID: call_5jTHskfgPCFQKakHl2dFxvsD
  Args:
    day: 2025-05-08
================================= Tool Message =================================

Available times on 2025-05-08: 9:00 AM, 2:00 PM, 4:00 PM
================================== Ai Message ==================================
Tool Calls:
  write_email (call_3tvSXyHFCWGk4yzbBYMCdEWz)
 Call ID: call_3tvSXyHFCWGk4yzbBYMCdEWz
  Args:
    to: partner@home.com
    subject: Re: Meet Jim and Lisa for brunch in 3 week

## Respond (with feedback) `write_email`, `schedule_meeting`, and `question`

Our final test set explores the "response" feedback pattern - providing guidance without directly editing or accepting. This conversational feedback mechanism offers a middle ground between acceptance and editing:

1. First, we'll test feedback for meeting scheduling by requesting:
   - Shorter duration (30 minutes instead of 45)
   - Afternoon meeting times (after 2pm)
   
2. Next, we'll test feedback for email drafting by requesting:
   - Shorter, less formal language
   - A specific closing statement about looking forward to the meeting
   
3. Finally, we'll test feedback for questions by providing:
   - A direct answer with additional context
   - Specific preferences (brunch location, time)

This natural language feedback approach lets users guide the assistant without having to do the work themselves. We expect to see detailed memory updates that extract the general principles from our specific feedback.

In [21]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_5 = uuid.uuid4()
thread_config_5 = {"configurable": {"thread_id": thread_id_5}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt 
display_memory_content(store, ("email_assistant", "cal_preferences"))

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}

======= CURRENT MEMORY CONTENT =======

--- cal_preferences ---
{'preferences': '\n30 minute meetings are preferred, but 15 minute meetings are also acceptable.\n'}


Provide feedback for the `schedule_meeting` tool call

Instead of directly editing the meeting proposal or simply accepting it, we'll provide natural language feedback:

1. We request a 30-minute meeting instead of 45 minutes
2. We express a preference for afternoon meetings after 2pm
3. The system must interpret this feedback and generate a new proposal

This conversational approach is often more natural and efficient than direct editing, especially for mobile users or those who prefer to give high-level direction rather than detailed edits.

After providing feedback, we'll examine the calendar preferences memory to see how this natural language guidance is captured. We expect to see the system extract both the meeting duration and time-of-day preferences as general principles.

In [22]:
print(f"\nSimulating user providing feedback for the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Please schedule this for 30 minutes instead of 45 minutes, and I prefer afternoon meetings after 2pm."}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after providing feedback for schedule_meeting
print("\nChecking memory after providing feedback for schedule_meeting:")
display_memory_content(store, ("email_assistant", "cal_preferences"))


Simulating user providing feedback for the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 30, 'preferred_day': '2025-04-22', 'start_time': 14}}

Checking memory after providing feedback for schedule_meeting:

======= CURRENT MEMORY CONTENT =======

--- cal_preferences ---
{'preferences': '30 minute meetings are preferred, but 15 minute meetings are also acceptable.\nAfternoon meetings after 2pm are preferred.'}


Our memory check after providing feedback shows an elegantly simple calendar preference update:

```
30 minute meetings are preferred, but 15 minute meetings are also acceptable.
Afternoon meetings after 2pm are preferred.
```

The system has:
1. Captured both aspects of our feedback (duration and time of day)
2. Preserved the existing preference about 15-minute meetings
3. Added our preference for afternoon meetings after 2pm as a new line
4. Kept the format clean and readable

This natural language feedback mechanism creates the same quality of memory updates as direct editing but requires less effort from the user. The system is able to extract structured preferences from unstructured feedback, showing its ability to learn from conversational interactions.

Let's accept this revised meeting proposal and move to the email draft:

In [23]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting schedule_meeting after feedback
print("\nChecking memory after accepting schedule_meeting after feedback:")
display_memory_content(store, ("email_assistant", "response_preferences"))


Simulating user accepting the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hello,\n\nThank you for reaching out regarding tax planning strategies. I am available on Tuesday, April 22nd at 2:00 PM, and have scheduled a 30-minute call for us to discuss your suggestions. If you need more time or would prefer a different slot, please let me know.\n\nLooking forward to our conversation.\n\nBest regards,\nLance'}}

Checking memory after accepting schedule_meeting after feedback:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (usi

Now provide feedback for the `write_email` tool call

Similar to our meeting feedback, we'll now provide natural language guidance for the email draft:

1. We request "shorter and less formal" language - a style preference
2. We ask for a specific closing statement about looking forward to the meeting
3. The system must interpret this guidance and rewrite the email accordingly

After providing this feedback, we'll check the response preferences memory to see how these style and structure preferences are captured. We expect to see generalizable guidelines about email brevity, formality, and closing statements added to our preference profile.

In [24]:
print(f"\nSimulating user providing feedback for the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Shorter and less formal. Include a closing statement about looking forward to the meeting!"}]), config=thread_config_5):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after providing feedback for write_email
print("\nChecking memory after providing feedback for write_email:")
display_memory_content(store, ("email_assistant", "response_preferences"))


Simulating user providing feedback for the write_email tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hi,\n\nThanks for reaching out. I’ve scheduled us for a 30-minute call on Tuesday, April 22nd at 2:00 PM to go over tax planning. Let me know if you need a different time.\n\nLooking forward to our chat!\n\nBest,\nLance'}}

Checking memory after providing feedback for write_email:

======= CURRENT MEMORY CONTENT =======

--- response_preferences ---
{'preferences': "Use professional and concise language. If the e-mail mentions a deadline, make sure to explicitly acknowledge and reference the deadline in your response.\n\nWhen responding to technical questions that require investigation:\n- Clearly state whether you will investigate or who you will ask\n- Provide an estimated timeline for when you'll have more information or complete the task\n\nWhen responding to eve

The memory update after our email feedback shows highly sophisticated learning about both meeting scheduling and email writing preferences:

1. The system has added a complete new section to the response preferences entitled "When writing email responses" with two key preferences:
   - "Favor shorter and less formal language when possible, unless the context requires formality"
   - "Include a closing statement expressing that you look forward to the meeting or conversation when confirming appointments"

2. It has also added a new bullet point to the "When responding to meeting scheduling requests" section:
   - "When scheduling meetings, prefer afternoon times after 2pm when possible, and default to 30-minute durations unless otherwise specified"

This demonstrates the system's ability to:
- Organize learned preferences into appropriate categories
- Extract multiple insights from a single feedback instance
- Apply meeting preferences to both calendar and email contexts
- Capture nuance with appropriate qualifiers ("when possible," "unless otherwise specified")
- Maintain the hierarchical structure of the memory

The resulting email shows all these preferences applied: it's shorter, less formal, includes a closing statement about looking forward to the chat, and correctly references the 30-minute meeting time.

In [25]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting write_email after feedback
print("\nChecking memory after accepting write_email after feedback:")
display_memory_content(store, ("email_assistant", "response_preferences"))


Simulating user accepting the write_email tool call...

Checking memory after accepting write_email after feedback:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR Department deadline reminders\n- Subscription status / renewal reminders\n- GitHub notifications\n\nEmails that are worth responding to:\n- Direct questions from team members requiring expertise\n- Meeting r

Look at the full message history.

In [26]:
state = graph.get_state(thread_config_5)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_SwwMW5F9zEWmajYTDLjKOoFN)
 Call ID: call_SwwMW5F9zEWmajYTDLjKOoFN
  Args:
    day: 2025-04-22
  check_calendar_availability (call_nIBAphQD4FFOdW89Ikc1uNfV)
 Call ID: call_nIBAphQD4FFOdW89Ikc1uNfV
  Args:
    day: 2025-04-24
================================= Tool Message =================================

Ava

Now let's try an email that calls the `Question` tool to provide feedback

Our final test examines feedback for questions. When the assistant needs clarification before proceeding, users can provide detailed information beyond just answering the question:

1. For the brunch invitation email, we'll provide feedback that includes:
   - Confirmation that we want to invite the people mentioned
   - A specific location preference (Jack's)
   - A time preference (before 11am)
   
2. This gives the system multiple pieces of information:
   - A direct answer to the question (yes, let's invite them)
   - Additional context and preferences not explicitly asked for

This tests the system's ability to process compound feedback and extract multiple data points from a single response.

In [27]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Partner <partner@home.com>",
    "subject": "Meet Jim and Lisa for brunch in 3 weeks?",
    "email_thread": "Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about."
}

# Compile the graph
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_6 = uuid.uuid4()
thread_config_6 = {"configurable": {"thread_id": thread_id_6}}

# Run the graph until the first interrupt
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_6):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt for Question tool
display_memory_content(store)

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'Question', 'args': {'content': 'Do you want me to check your calendar and suggest specific dates and times for the brunch, or would you like to propose a particular day and time to Jim and Lisa first?'}}

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant inform

Provide feedback for the `Question` tool call

When the assistant asks about our preferences for the brunch invitation, we'll respond with rich, multi-faceted feedback:

1. We confirm we want to invite Jim and Lisa
2. We specify a location preference (Jack's, not the place on 17th)
3. We express a time preference (before 11am)

This tests the system's ability to handle compound responses that both answer the direct question and provide additional context. Note that we're not just answering "yes" or "no" - we're providing a rich context that should influence the assistant's next actions.

An ideal system would use this feedback to both respond to the immediate email and update background knowledge that could be relevant for future similar social invitations. In our current implementation, we don't update background knowledge from question responses, but this would be a straightforward enhancement.

In [28]:
print(f"\nSimulating user providing feedback for the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Yes, let's invite them, I really like brunch at Jack's, ideally before 11am."}]), config=thread_config_6):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after providing feedback for Question
print("\nChecking memory after providing feedback for Question:")
display_memory_content(store)


Simulating user providing feedback for the Question tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'partner@home.com', 'subject': 'Brunch with Jim and Lisa in 3 Weeks', 'content': 'Great idea! I’ll reach out to Jim and Lisa to invite them to brunch at Jack’s in three weeks. I’ll suggest Saturday, May 8th, with a start time before 11am—9:00 AM is available on our calendar. Let me know if you’d like to suggest a different time or if this works for you before I send the invite.'}}

Checking memory after providing feedback for Question:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples 

Currently, we don't update `background_information` when the user provides feedback for the `Question` tool, but this would be a valuable enhancement.

Looking at how the system handled our question response:

1. It correctly incorporated all three key pieces of information:
   - Our affirmative decision to invite Jim and Lisa
   - Our location preference (Jack's, not the place on 17th)
   - Our time preference (before 11am)

2. It drafted a complete email that:
   - References reaching out to Jim and Lisa
   - Specifies Jack's as the location
   - Suggests a 9:00 AM time (before 11am as requested)
   - Asks for confirmation before sending an invite

This demonstrates the system's ability to extract and use detailed information from natural language feedback, even when not explicitly updating memory. The email correctly incorporates all aspects of our feedback and presents a coherent plan based on our preferences.

A future enhancement could store these preferences (location preferences, time preferences for social events) in the background information memory for use in future similar situations.

In [29]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_6):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting write_email after answering Question
print("\nChecking memory after accepting write_email after answering Question:")
display_memory_content(store)


Simulating user accepting the write_email tool call...
================================== Ai Message ==================================
Tool Calls:
  Done (call_k6NQ6GqqEIE1uNwVQXmx6xU6)
 Call ID: call_k6NQ6GqqEIE1uNwVQXmx6xU6
  Args:
    done: True

Checking memory after accepting write_email after answering Question:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR De

Look at the full message history

In [30]:
state = graph.get_state(thread_config_6)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Meet Jim and Lisa for brunch in 3 weeks?
**From**: Partner <partner@home.com>
**To**: Lance Martin <lance@company.com>

Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about.

---

================================== Ai Message ==================================
Tool Calls:
  Question (call_dF0WQiJIDf5ovo5veMixbBm9)
 Call ID: call_dF0WQiJIDf5ovo5veMixbBm9
  Args:
    content: Do you want me to check your calendar and suggest specific dates and times for the brunch, or would you like to propose a particular day and time to Jim and Lisa first?
================================= Tool Message =================================

User answered the question, which can we can use for any follow up actions. Feedback: Yes, let's invite them, I really like brunch at Jack's, ideally before 11am.
==================

## Test Case for Notify Classification

This test explores how memory updates when an email is initially classified as "NOTIFY" but the user decides it needs a response:

1. The triage system initially classifies IT security updates as notifications
2. But the user decides this particular notification warrants acknowledgment
3. This creates a learning opportunity about which notification types need responses

The "notify" category is designed for important information that doesn't need immediate action. But user feedback can help the system learn which subset of notifications actually do warrant responses, refining the initial classification over time.

In [34]:
# Notify - Important FYI Email
email_input_notify = {
    "to": "Team Members <team@company.com>",
    "author": "IT Department <it@company.com>",
    "subject": "Critical Security Update",
    "email_thread": "Dear Team,\n\nThis is an important security notification. We will be updating our authentication system this weekend. During the update window (Saturday 2am-4am), you will not be able to access company resources.\n\nPlease ensure you log out of all systems before the maintenance window.\n\nRegards,\nIT Department"
}

# Compile the graph with new thread
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_7 = uuid.uuid4()
thread_config_7 = {"configurable": {"thread_id": thread_id_7}}

# Run the graph until the first interrupt - should be classified as "notify"
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_notify}, config=thread_config_7):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt for Notify
display_memory_content(store, ("email_assistant", "triage_preferences"))

Running the graph until the first interrupt...
🔔 Classification: NOTIFY - This email contains important information

INTERRUPT OBJECT:
Action Request: {'action': 'Email Assistant: notify', 'args': {}}

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR Department deadline reminders\n- Subscription status / renewal reminders\n- GitHub notifications\n\nEmails that are worth r

Now simulate user deciding to respond with feedback.

This test explores the reclassification pathway from "notify" to "respond":

1. The system initially classifies a security update as information-only ("notify")
2. We're presented with this notification without a suggested action
3. We decide this security update actually requires acknowledgment
4. We provide feedback indicating we want to respond and confirm our compliance

This represents an important learning signal - that security notifications, particularly those requesting specific user actions, should be treated as items requiring response rather than just information.

After providing this feedback, we'll check the triage preferences memory to see how this "override" affects future classifications. We expect to see security notifications with action requests moved into the "worth responding to" section.

In [35]:
print("\nSimulating user deciding to respond with feedback...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "We should acknowledge receipt of this important notice and confirm that we'll be logged out before the maintenance window."}]), config=thread_config_7):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after responding with feedback to Notify
print("\nChecking memory after responding with feedback to Notify:")
display_memory_content(store, ("email_assistant", "triage_preferences"))


Simulating user deciding to respond with feedback...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'it@company.com', 'subject': 'Re: Critical Security Update', 'content': 'Dear IT Department,\n\nThank you for the notification regarding the upcoming authentication system update. I acknowledge receipt of this important notice and will ensure that I am logged out of all systems before the maintenance window begins on Saturday from 2am to 4am.\n\nBest regards,\nLance'}}

Checking memory after responding with feedback to Notify:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "Emails that are worth responding to:\n- Direct questions from team members requiring expertise\n- Meeting requests requiring confirmation\n- Critical bug reports related to team's projects\n- Requests from management requiring acknowledgment\n- Client inquiries about project status or features\n- Technical questions about documentation, code, or APIs (

The memory update after our decision to respond to a security notification shows a remarkable triage preference refinement:

1. The system has added a new bullet to the "Emails that are worth responding to" section:
   ```
   "Important security notifications from IT Department requiring acknowledgment or confirmation of action"
   ```

2. This update demonstrates:
   - Precision: It specifically identifies "security notifications" (not all IT emails)
   - Source awareness: It's from the "IT Department" specifically
   - Action trigger: It identifies notifications "requiring acknowledgment or confirmation"
   - Category reorganization: It moves this from "notify" to "respond" without removing notifications entirely

The system didn't just record our specific override - it analyzed the *reason* we might want to respond (acknowledging required action) and created a generalizable rule. This would ensure that future similar security notifications requesting specific user actions would be correctly classified as needing response.

In [36]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_7):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after accepting write_email for Notify
print("\nChecking memory after accepting write_email for Notify:")
display_memory_content(store, ("email_assistant", "triage_preferences"))


Simulating user accepting the write_email tool call...

Checking memory after accepting write_email for Notify:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "Emails that are worth responding to:\n- Direct questions from team members requiring expertise\n- Meeting requests requiring confirmation\n- Critical bug reports related to team's projects\n- Requests from management requiring acknowledgment\n- Client inquiries about project status or features\n- Technical questions about documentation, code, or APIs (especially questions about missing endpoints or features)\n- Personal reminders related to family (wife / daughter)\n- Personal reminder related to self-care (doctor appointments, etc)\n- Important security notifications from IT Department requiring acknowledgment or confirmation of action\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). E

Look at the full message history.

In [37]:
state = graph.get_state(thread_config_7)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Email to notify user about: 

**Subject**: Critical Security Update
**From**: IT Department <it@company.com>
**To**: Team Members <team@company.com>

Dear Team,

This is an important security notification. We will be updating our authentication system this weekend. During the update window (Saturday 2am-4am), you will not be able to access company resources.

Please ensure you log out of all systems before the maintenance window.

Regards,
IT Department

---

================================ Human Message =================================

User wants to reply to the email. Use this feedback to respond: We should acknowledge receipt of this important notice and confirm that we'll be logged out before the maintenance window.
================================== Ai Message ==================================
Tool Calls:
  write_email (call_8tZq4eSFWr2WE01sPgEfgFr0)
 Call ID: call_8tZq4eSFWr2WE01sPgEfgFr0
  Args

## Test Case for Notify + Ignore

Our final test explores the opposite pattern - when a user decides notifications don't even warrant being shown:

1. The system classifies a company picnic announcement as a notification
2. The user decides this doesn't even warrant notification status
3. This creates a signal to further refine the classification boundaries

By ignoring certain types of notifications, users can teach the system which information is truly important to them versus which can be filtered out entirely. This completes the full spectrum of classification refinement.

In [39]:
# Notify - Important FYI Email
email_input_notify = {
    "to": "Team Members <team@company.com>",
    "author": "HR Department <hr@company.com>",
    "subject": "Company Picnic Next Month",
    "email_thread": "Dear Team,\n\nWe're planning the annual company picnic for next month. The tentative date is Saturday, June 15th from noon to 4pm at Central Park. There will be food, games, and activities for families.\n\nMore details will follow in the coming weeks.\n\nRegards,\nHR Department"
}

# Compile the graph with new thread
checkpointer = MemorySaver()
store = InMemoryStore()
graph = overall_workflow.compile(checkpointer=checkpointer, store=store)
thread_id_8 = uuid.uuid4()
thread_config_8 = {"configurable": {"thread_id": thread_id_8}}

# Run the graph until the first interrupt - should be classified as "notify"
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_notify}, config=thread_config_8):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after first interrupt for Notify + Ignore
display_memory_content(store, ("email_assistant", "triage_preferences"))

Running the graph until the first interrupt...
🔔 Classification: NOTIFY - This email contains important information

INTERRUPT OBJECT:
Action Request: {'action': 'Email Assistant: notify', 'args': {}}

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR Department deadline reminders\n- Subscription status / renewal reminders\n- GitHub notifications\n\nEmails that are worth r

Now simulate user deciding to ignore the notification.

In this final test, we explore the downgrade pathway from "notify" to "ignore":

1. The system initially classifies a company picnic announcement as "notify"
2. We decide we don't even want to be notified about these social events
3. By choosing "ignore," we signal this entire category should be filtered out

This represents another important learning signal - that certain types of company announcements (particularly social events) shouldn't even be surfaced as notifications, further refining our triage preferences.

After ignoring, we'll check the triage preferences memory for the final time to see how this override affects future classifications. We expect to see company social events moved from the "notify" section to the "not worth responding to" section.

In [40]:
print("\nSimulating user deciding to ignore the notification...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_8):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

# Check memory after ignoring Notify
print("\nChecking memory after ignoring Notify:")
display_memory_content(store, ("email_assistant", "triage_preferences"))


Simulating user deciding to ignore the notification...

Checking memory after ignoring Notify:

======= CURRENT MEMORY CONTENT =======

--- triage_preferences ---
{'preferences': "\nEmails that are not worth responding to:\n- Marketing newsletters and promotional emails\n- Spam or suspicious emails\n- CC'd on FYI threads with no direct questions\n- Company social event announcements (e.g., company picnic)\n\nThere are also other things that should be known about, but don't require an email response. For these, you should notify (using the `notify` response). Examples of this include:\n- Team member out sick or on vacation\n- Build system notifications or deployments\n- Project status updates without action items\n- Important company announcements\n- FYI emails that contain relevant information for current projects\n- HR Department deadline reminders\n- Subscription status / renewal reminders\n- GitHub notifications\n\nEmails that are worth responding to:\n- Direct questions from team 

The final memory update completes our triage preference refinement journey:

1. The system has added "Company social event announcements (e.g., company picnic)" to the "Emails not worth responding to" section

2. This demonstrates:
   - Content-based categorization: It identifies "social event announcements" specifically
   - Example inclusion: It provides an example "(company picnic)" for clarity
   - Category downgrade: It moves this from "notify" to "not worth responding to"
   - Structural preservation: It maintains the original memory organization

Through our series of tests, we've now explored the full spectrum of triage learning:
- Upgrading notifications to responses (security updates)
- Downgrading notifications to ignores (company social events)
- Downgrading responses to ignores (personal social invitations, routine calls)
- Refining response preferences (meeting duration, timing, email style)

Each interaction has produced targeted, intelligent memory updates that maintain the overall structure while adding new insights. This continuous refinement creates an increasingly personalized assistant that learns and adapts to user preferences over time.

In [41]:
state = graph.get_state(thread_config_8)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Email to notify user about: 

**Subject**: Company Picnic Next Month
**From**: HR Department <hr@company.com>
**To**: Team Members <team@company.com>

Dear Team,

We're planning the annual company picnic for next month. The tentative date is Saturday, June 15th from noon to 4pm at Central Park. There will be food, games, and activities for families.

More details will follow in the coming weeks.

Regards,
HR Department

---

================================ Human Message =================================

The user decided to ignore the email even though it was classified as notify. Update triage preferences to capture this.


## Testing with Local Deployment

You can find this graph with memory integration in the `src/email_assistant` directory:

* `src/email_assistant/email_assistant_hitl_memory.py`

Testing this locally gives you the full experience of a memory-enabled HITL system:

1. **Start the local server**: Run `langgraph dev` to launch the agent locally
2. **Connect Agent Inbox**: Use the graph URL from the `langgraph.json` file
3. **Submit test emails**: Try different email types to see classification in action
4. **Provide various feedback types**: Try accepting, editing, ignoring, and responding
5. **Observe memory evolution**: Check the Memory tab in LangGraph Studio to see changes

![inbox](img/agent-inbox-edit.png)

The Memory tab in LangGraph Studio offers a real-time view of how your preferences are being captured and updated with each interaction:

![studio-img](img/memory-studio.png)

Through continued use, the system becomes increasingly personalized:
- It learns which emails you want to respond to, be notified about, or ignore
- It adapts to your communication style preferences
- It remembers your scheduling preferences
- It refines its understanding with each interaction

This combination of HITL and memory creates a system that balances automation with control - handling routine tasks automatically while learning from your feedback to become more aligned with your preferences over time.

In [ ]:
! langgraph dev

![inbox](img/agent-inbox-edit.png)

As you provide feedback or edit replies, you can see memories accumulate in the `memory` tab in LangGraph Studio.

![studio-img](img/memory-studio.png)





### Use LangMem to add background memory! 

Add this to the `llm_call` node:

```
    # Search for existing background memory
    # TODO: Here, semantic search over a facts collection of background information from emails could be added. 
    # background = get_memory(store, ("email_assistant", "background"), default_background)
```

Add this to the interrupt handler node:

```
elif tool_call["name"] == "Question":
    # Don't execute the tool, and add a message with the user feedback to incorporate into the email
    result.append({"role": "tool", "content": f"User answered the question, which can we can use for any follow up actions. Feedback: {user_feedback}", "tool_call_id": tool_call["id"]})
    # TODO: Here, we could update the background information with the user's answer. 
    # update_memory(store, ("email_assistant", "background"), [{
    #     "role": "user",
    #     "content": f"Update background information based upon these messages:"
    # }] + state["messages"] + result)
```

Consider using LangMem: 
https://langchain-ai.github.io/langmem/